In [13]:
import librosa
import numpy as np
import scipy
from sklearn.neighbors import NearestNeighbors
import pydub
import math

In [14]:
songs_wav = ["furelise1.wav", "furelise2.wav", "furelise3.wav", "furelise4.wav"]

In [15]:
original, sample_rate = librosa.load("furelise.wav")

In [26]:
song_data = [original] + [librosa.load(song)[0] for song in songs_wav]

In [27]:
def remove_zeros(vec):
    temp = np.transpose(vec == 0)
    indices = np.argwhere(temp == False)
    return vec[indices[0][0]:indices[len(indices) - 1][0]]

In [29]:
songs = [remove_zeros(s) for s in song_data] 

In [79]:
fft_frame_size = 2000

In [80]:
def get_fft_chunks(time_data):
    num_samples= len(time_data)//fft_frame_size
    return [np.fft.fft(time_data[i*fft_frame_size:(i+1)*fft_frame_size]) for i in range(num_samples)]

In [81]:
original_fft = get_fft_chunks(remove_zeros(songs[0]))

In [82]:
#testing the stackoverflow code
#2000 is the length of one window
freq = np.fft.fftfreq(fft_frame_size)
fft_freqs = [abs(freq[i]*sample_rate) for i in range(fft_frame_size)]

In [83]:
ranges = [40, 80, 120, 180, 300]

In [87]:
def get_magnitudes(fft):
    high_mags = [np.zeros(len(ranges)) for i in range(len(fft))]
    for fft_window in range(len(fft)):
        window = fft[fft_window]
        i = 0
        mag = (0,0)
        while(fft_freqs[i] < 40): 
            curr_mag = math.log10(abs(window[i])+1)
            if curr_mag > mag[0]:
                mag = (curr_mag, i)
            i += 1
        high_mags[fft_window][0] = fft_freqs[mag[1]]
        mag = (0,0)
        while(fft_freqs[i] < 80):
            curr_mag = math.log10(abs(window[i])+1)
            if curr_mag > mag[0]:
                mag = (curr_mag, i)
            i+=1
        high_mags[fft_window][1] = fft_freqs[mag[1]]
        mag = (0,0)
        while(fft_freqs[i] < 120):
            curr_mag = math.log10(abs(window[i])+1)
            if curr_mag > mag[0]:
                mag = (curr_mag, i)
            i+=1
        high_mags[fft_window][2] = fft_freqs[mag[1]]
        mag = (0,0)
        while(fft_freqs[i] < 180):
            curr_mag = math.log10(abs(window[i])+1)
            if curr_mag > mag[0]:
                mag = (curr_mag, i)
            i+=1
        high_mags[fft_window][3] = fft_freqs[mag[1]]
        mag = (0,0)
        while(fft_freqs[i] < 300):
            curr_mag = math.log10(abs(window[i])+1)
            if curr_mag > mag[0]:
                mag = (curr_mag, i)
            i+=1
        high_mags[fft_window][4] = fft_freqs[mag[1]]
    return high_mags

In [91]:
mags = get_magnitudes(original_fft)
mags

[array([  33.075,   44.1  ,   88.2  ,  121.275,  264.6  ]),
 array([  33.075,   77.175,   88.2  ,  132.3  ,  286.65 ]),
 array([  33.075,   77.175,   88.2  ,  132.3  ,  198.45 ]),
 array([  33.075,   44.1  ,   88.2  ,  132.3  ,  286.65 ]),
 array([  33.075,   77.175,   88.2  ,  132.3  ,  209.475]),
 array([  33.075,   44.1  ,   88.2  ,  132.3  ,  275.625]),
 array([  11.025,   77.175,   88.2  ,  176.4  ,  198.45 ]),
 array([  11.025,   77.175,   99.225,  132.3  ,  264.6  ]),
 array([  33.075,   44.1  ,   88.2  ,  121.275,  187.425]),
 array([  33.075,   44.1  ,   88.2  ,  132.3  ,  264.6  ]),
 array([  33.075,   44.1  ,  110.25 ,  121.275,  264.6  ]),
 array([  22.05 ,   44.1  ,  110.25 ,  121.275,  253.575]),
 array([  33.075,   44.1  ,   88.2  ,  132.3  ,  253.575]),
 array([  22.05 ,   55.125,   88.2  ,  132.3  ,  242.55 ]),
 array([  11.025,   44.1  ,   88.2  ,  121.275,  264.6  ]),
 array([  22.05 ,   77.175,   99.225,  121.275,  297.675]),
 array([  33.075,   44.1  ,  110.25 ,  1

In [ ]:
database = {} #should be code -> list of tuples of type (time, song)
#where time is the "window" and song is a string such as "furelise
#populating the database with original fur elise data

for i in range(len(mags)): #i = index of "window" so it corresponds to what we can consider "time" i suppose
    key = str(mags[i])
    if key not in database:
        database[key] = []
    database[key].append((i,"furelise"))
    